In [1]:
import os
import re
import pandas as pd
import numpy as np
from collections import Counter
from itertools import combinations

In [2]:
import json
from collections import defaultdict

In [3]:
# Scripts like: **Phoebe shakes her hand and says: Phoe-Be.**
ACTION_WORDS = {
    "says", "shakes", "walks", "laughs", "enters", "exits",
    "shouts", "yells", "cries", "looks", "stares", "turns",
    "moves", "explains", "asks", "replies", "continues",
    "tells", "answers", "runs", "steps", "smiles", "nods", 
    "starts", "storms", "coming", "starting", "raises", 
    "have", "screaming", "to", "cutting", "at", "with", 
    "opens", "looking", "has", "gets"
}

In [4]:
GROUP_SPEAKER_PHRASES = {
    "all", "everyone", "guys", "both", 
    "the guys", "girls", "girl", "boys", "boy", 
    "guy", "women", "woman", "men", "man"
}

In [5]:
GROUP_INDICATORS = {
    "girls", "girl", "boys", "boy", "guys",  
    "guy", "crowd", "people", "kids", "kid", "friend", 
    "friends", "family", "parents", "neighbors", 
    "roommates", "siblings", "child", "children", 
    "women", "men"
}

In [6]:
GENERIC_ROLES = {
    "customer", "customers", "receptionist", 
    "receptionists", "nurse", "waiter", "waiters", 
    "waitress", "teacher", "clerk"
}

In [7]:
def split_multi_speaker(raw):
    """
    Deal with speaker names like Joey And Chandler.
    """
    raw = raw.strip().lower()
    
    if raw in GROUP_SPEAKER_PHRASES:
        return []
    
    raw_norm = re.sub(r"\band\b|&", ",", raw, flags=re.IGNORECASE)
    parts = [p.strip().title() for p in raw_norm.split(",") if p.strip()]

    return parts

In [8]:
def parse_episode_file(file_path):
    """
    Parse a single Friends script file to extract scene-based character interactions.

    Parameters
    ----------
    file_path : str
        Path to the .txt file for a single episode (e.g. "S01E01 Monica Gets A Roommate.txt")

    Returns
    -------
    scenes : list[set[str]]
        List of sets, where each set contains all characters appearing in one scene.
        Example: [{'Monica', 'Ross', 'Chandler'}, {'Rachel', 'Phoebe', 'Monica'}, ...]

    interactions : dict[(str, str), int]
        Dictionary of pairwise co-occurrence counts across all scenes.
        Example: {('Monica','Ross'): 8, ('Monica','Chandler'): 5, ...}
    """

    scenes = []
    current_scene = set()
    
    MAIN_CHARS = ["Monica", "Rachel", "Phoebe", "Ross", "Chandler", 
                    "Joey", "Ursula", "Carol", "Susan", "Janice"
    ]

    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()

    for line in text.splitlines():
        line = line.strip()
        
        if not line:
            continue
        
        # Skip script information. 
        if line.lower().startswith(("written by", "story by", "teleplay by")): # e.g. Written by: David Crane
            continue
        
        # Action Direction. e.g. (They all stare, bemused.)
        if line.startswith("("): 
            continue
        
        # Change scenes. 
        if re.match(r"^\[(Scene|Time|Cut|Commercial|Closing)", line):
            # Append the latest scene set into the scenes list.
            if current_scene:
                scenes.append(current_scene)
            # Initialize a new scene set.
            current_scene = set()
            continue

        # Normal match e.g. Monica: blah blah blah
        # Mrs. Geller: blah blah blah
        normal_match = re.match(r"^([A-Z][a-zA-Z\s.']+)(?=[:(])", line)
        if normal_match:
            prefix = normal_match.group(1).strip()
            lower_prefix = prefix.lower()
            
            if lower_prefix in GROUP_SPEAKER_PHRASES: # E.g. all, everyone.
                continue
            if lower_prefix in GENERIC_ROLES: # E.g. customer, teacher.
                continue
            
            # Exclude phrases like Phoebe's Friends
            words_no_ap = re.sub(r"'", "", lower_prefix).split()
            if any(w in GROUP_INDICATORS for w in words_no_ap):
                continue
            
            if any(w in ACTION_WORDS for w in words_no_ap):
                pass # Go to fallback. 
            else:            
                speakers = split_multi_speaker(prefix)
                for speaker in speakers:
                    current_scene.add(speaker)
                continue
        
        # Fallback. Deal with cases like: 
        # Phoebe shakes her hand and says: Phoe-Be. 
        if ":" in line:
            prefix = line.split(":", 1)[0].strip()
            lower_prefix = prefix.lower()
            
            if lower_prefix in GROUP_SPEAKER_PHRASES:
                continue
            if lower_prefix in GENERIC_ROLES:
                continue
            
            words_no_ap = re.sub(r"'", "", lower_prefix).split()
            if any(w in GROUP_INDICATORS for w in words_no_ap):
                continue
            
            fallback = [name for name in MAIN_CHARS if name.lower() in lower_prefix]
            if len(fallback) == 1:
                current_scene.add(fallback[0])
                continue
            else:
                # No match. Just continue.
                continue
        
        # Nothing match, just continue. 
        continue


    if current_scene:
        scenes.append(current_scene)

    interaction_counter = Counter()
    for scene in scenes:
        for a, b in combinations(sorted(scene), 2):
            interaction_counter[(a, b)] += 1

    return scenes, dict(interaction_counter)


In [9]:
def parse_all_scripts(folder_path):
    """
    Batch-parse all Friends script files in a folder.

    Parameters
    ----------
    folder_path : str
        Path to folder containing all .txt episode files.

    Returns
    -------
    all_edges_df : pandas.DataFrame
        DataFrame with columns [episode_id, source, target, weight].
        Each row represents an interaction between two characters within an episode.

    episode_summary : dict[str, dict]
        Meta info per episode: number of scenes, unique characters, total interactions.
    """

    all_edges = []
    episode_summary = {}

    for file in sorted(os.listdir(folder_path)):
        if not file.endswith(".txt"):
            continue

        file_path = os.path.join(folder_path, file)

        match = re.search(r"S\d+E\d+(?:-S?\d*E?\d+)?", file, flags=re.IGNORECASE)
        if not match:
            print(f"Skipping {file} (no episode id found)")
            continue
        episode_id = match.group().upper()

        scenes, interactions = parse_episode_file(file_path)

        for (a, b), w in interactions.items():
            all_edges.append({
                "episode_id": episode_id,
                "source": a,
                "target": b,
                "weight": w
            })

        episode_summary[episode_id] = {
            "file": file,
            "num_scenes": len(scenes),
            "num_characters": len(set().union(*scenes)),
            "num_edges": len(interactions)
        }

    all_edges_df = pd.DataFrame(all_edges)
    print(f"Parsed {len(episode_summary)} episodes, total {len(all_edges_df)} edges.")
    return all_edges_df, episode_summary

In [10]:
scripts_folder = "./Scripts"
edges_df, episode_meta = parse_all_scripts(scripts_folder)


Parsed 228 episodes, total 6960 edges.


In [11]:
edges_df.to_csv("edge_1113.csv")


In [12]:
edges_df.head()

,episode_id,source,target,weight
0,S01E01,Chandler,Joey,10
1,S01E01,Chandler,Monica,9
2,S01E01,Chandler,Phoebe,7
3,S01E01,Joey,Monica,8
4,S01E01,Joey,Phoebe,6


In [20]:
episode_meta

{'S01E01': {'file': 'S01E01 Monica Gets A Roommate.txt',
  'num_scenes': 16,
  'num_characters': 9,
  'num_edges': 23},
 'S01E02': {'file': 'S01E02 The Sonogram At The End.txt',
  'num_scenes': 14,
  'num_characters': 15,
  'num_edges': 34},
 'S01E03': {'file': 'S01E03 The Thumb.txt',
  'num_scenes': 15,
  'num_characters': 9,
  'num_edges': 22},
 'S01E04': {'file': 'S01E04 George Stephanopoulos.txt',
  'num_scenes': 16,
  'num_characters': 9,
  'num_edges': 36},
 'S01E05': {'file': 'S01E05 The East German Laundry Detergent.txt',
  'num_scenes': 17,
  'num_characters': 9,
  'num_edges': 26},
 'S01E06': {'file': 'S01E06 The Butt.txt',
  'num_scenes': 9,
  'num_characters': 8,
  'num_edges': 22},
 'S01E07': {'file': 'S01E07 The Blackout.txt',
  'num_scenes': 23,
  'num_characters': 9,
  'num_edges': 19},
 'S01E08': {'file': 'S01E08 Nana Dies Twice.txt',
  'num_scenes': 14,
  'num_characters': 12,
  'num_edges': 42},
 'S01E09': {'file': 'S01E09 Underdog Gets Away.txt',
  'num_scenes': 14,

In [27]:
def save_parsed_data_as_json(scripts_folder, output_folder):
    """
    Batch-parse all Friends script files in `scripts_folder`,
    and save the structured data as JSON into `output_folder`.

    Output JSON files:
        - episode_scenes.json
        - episode_interactions.json
        - episode_characters.json
        - episode_meta.json
    """

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    episode_scenes = {}          # episode_id -> list of scenes (each scene is a list)
    episode_interactions = {}    # episode_id -> { "A-B": weight }
    episode_characters = {}      # episode_id -> list of unique characters
    episode_meta = {}            # episode_id -> metadata (scenes, chars, etc.)

    # Collect all .txt scripts
    script_files = sorted(f for f in os.listdir(scripts_folder) if f.endswith(".txt"))

    for file in script_files:
        file_path = os.path.join(scripts_folder, file)

        # Extract episode_id from filename (e.g., S02E12-S02E13)
        match = re.search(r"S\d+E\d+(?:-S?\d*E?\d+)?", file, flags=re.IGNORECASE)
        if not match:
            print(f"⚠️  Skipping file (no episode id found): {file}")
            continue

        episode_id = match.group().upper()

        # Parse the episode
        scenes, interactions = parse_episode_file(file_path)

        # Prepare structures
        episode_scenes[episode_id] = [sorted(list(scene)) for scene in scenes]

        # Convert (A,B) tuples → "A-B" JSON-friendly keys
        inter_json = {}
        for (a, b), w in interactions.items():
            key = f"{a}-{b}"
            inter_json[key] = w
        episode_interactions[episode_id] = inter_json

        # characters list (union of all scenes)
        char_set = set()
        for scene in scenes:
            char_set |= scene
        episode_characters[episode_id] = sorted(list(char_set))

        # meta info
        episode_meta[episode_id] = {
            "file": file,
            "num_scenes": len(scenes),
            "num_characters": len(char_set),
            "num_edges": len(interactions)
        }

        print(f"✓ Parsed {episode_id}  | scenes: {len(scenes)}, chars: {len(char_set)}, edges: {len(interactions)}")
        
        def save_json(data, name):
            path = os.path.join(output_folder, name)
            with open(path, "w", encoding="utf-8") as f:
                json.dump(data, f, indent=4, ensure_ascii=False)
            print(f"💾 Saved {name} ({len(data)} items)")
        
        save_json(episode_scenes, "episode_scenes.json")
        save_json(episode_interactions, "episode_interactions.json")
        save_json(episode_characters, "episode_characters.json")
        save_json(episode_meta, "episode_meta.json")

        print("\n🎉 All JSON cache files generated successfully!")
    


In [28]:
save_parsed_data_as_json(
    scripts_folder="./Scripts",
    output_folder="./Friends_json"
)

✓ Parsed S01E01  | scenes: 16, chars: 9, edges: 23
💾 Saved episode_scenes.json (1 items)
💾 Saved episode_interactions.json (1 items)
💾 Saved episode_characters.json (1 items)
💾 Saved episode_meta.json (1 items)

🎉 All JSON cache files generated successfully!
✓ Parsed S01E02  | scenes: 14, chars: 15, edges: 34
💾 Saved episode_scenes.json (2 items)
💾 Saved episode_interactions.json (2 items)
💾 Saved episode_characters.json (2 items)
💾 Saved episode_meta.json (2 items)

🎉 All JSON cache files generated successfully!
✓ Parsed S01E03  | scenes: 15, chars: 9, edges: 22
💾 Saved episode_scenes.json (3 items)
💾 Saved episode_interactions.json (3 items)
💾 Saved episode_characters.json (3 items)
💾 Saved episode_meta.json (3 items)

🎉 All JSON cache files generated successfully!
✓ Parsed S01E04  | scenes: 16, chars: 9, edges: 36
💾 Saved episode_scenes.json (4 items)
💾 Saved episode_interactions.json (4 items)
💾 Saved episode_characters.json (4 items)
💾 Saved episode_meta.json (4 items)

🎉 All JSON